In [1]:
import numpy as np
import pandas as pd
import urllib
import json

with open(".key", "r") as f:
    key = f.read()

In [2]:
uzbekBBC = 'UCGu3z18dm6hqQAScxG12qBg'
farsiBBC = 'UCHZk9MrT3DGWmVqdsj5y0EA'

In [20]:
url = ('https://www.googleapis.com/youtube/v3/search?channelId={}&key={}'.format(uzbekBBC, key)
      + '&maxResults=50')

with urllib.request.urlopen(url) as f:
    searchResult = f.read().decode('utf-8')
    print(searchResult)

{
  "kind": "youtube#searchListResponse",
  "etag": "9MHyKLzQkyZxcw7-tultMhEaUOw",
  "nextPageToken": "CDIQAA",
  "regionCode": "US",
  "pageInfo": {
    "totalResults": 1473,
    "resultsPerPage": 50
  },
  "items": [
    {
      "kind": "youtube#searchResult",
      "etag": "z1vllV_ORSlJbPgaAsp8cHTV6m4",
      "id": {
        "kind": "youtube#video",
        "videoId": "0ON22otQYVc"
      }
    },
    {
      "kind": "youtube#searchResult",
      "etag": "XvijdrILozhKI2NqJ69I_r6CpIs",
      "id": {
        "kind": "youtube#video",
        "videoId": "kWTFrgvTA84"
      }
    },
    {
      "kind": "youtube#searchResult",
      "etag": "ZoYbwSvY3d1i1Muxm3FX3hJyaf0",
      "id": {
        "kind": "youtube#video",
        "videoId": "6ivFBWKX1B0"
      }
    },
    {
      "kind": "youtube#searchResult",
      "etag": "SeXbWFVgg87zcJK1pEJvggsYz2c",
      "id": {
        "kind": "youtube#video",
        "videoId": "AWfaWVi8480"
      }
    },
    {
      "kind": "youtube#searchResult",
 

In [29]:
videoIds = ','.join(item['id']['videoId'] 
                    for item in json.loads(searchResult)['items'] if item['id']['kind'] == 'youtube#video')
videoIds

'0ON22otQYVc,kWTFrgvTA84,6ivFBWKX1B0,AWfaWVi8480,DzOkDzfzg8s,0dny7x5Ystc,hZPv3LgYzlE,_LpENg_QVto,w4FSipkuEXc,fkCD_jBIQQA,fU0dehOTexI,_ucGHelytU4,qsvPE4bocvU,38zhgxnl5yk,LfMn3UHAIhs,ubf27zBr-Bs,sSsOxjR1hAQ,nzPzikCcmcc,QAaaW8gd0IU,jpXKByHV0z0,MGWpgngN-i0,HkAlFFhhg1M,0zvS89y0C88,1V9TfNYeGhs,3b7rleHAzVQ,G4MCnvMy1qE,cjtx5l5nozk,l2XWaFnqoio,0oOAwNY09bM,W1nRd4RLVeI,HDJ7BSjmekQ,7eZq5dYR_ss,nzqMRfo5t6Q,GgVzRj8TOd4,t4n3aG1PIfw,e31hGv-hvTk,B_On4maOIJ0,geBXFq6YHNY,6Odw5lPhKL0,CFZdI61YEHQ,IXYgulkFIFQ,IUwUynkCfeE,cGUWLtc4rGg,Cq7VKD5Bmkc,tZ4iS36CWqM,iTzh5m9x4-Q,zeg3HeGKHkE,bieLcJwhakE'

In [13]:
url = ('https://www.googleapis.com/youtube/v3/videos?id={}&key={}&maxResults=50'.format(videoIds, key)
      + '&part=snippet,contentDetails')

with urllib.request.urlopen(url) as f:
    videoResult = f.read().decode('utf-8')
    videoJson = json.loads(videoResult)
    print(videoResult)

{
  "kind": "youtube#videoListResponse",
  "etag": "xPYlBuBf5NJlWSoSHyKdywzTZ-Y",
  "items": [
    {
      "kind": "youtube#video",
      "etag": "A46KEwzWNglSeYD3PQD0IP1wMrc",
      "id": "0ON22otQYVc",
      "snippet": {
        "publishedAt": "2019-02-25T15:03:46Z",
        "channelId": "UCGu3z18dm6hqQAScxG12qBg",
        "title": "طالب لر بیلن مذاکره لر دن افغانستان لیک عیاللری تشویشده",
        "description": "بی بی سی اۉزبېک - اوزبیک  تلویزیون دستورلرینی ساعت۶ ده آرزو تلویزیونی آرقه لی کۉریشینگیزممکن",
        "thumbnails": {
          "default": {
            "url": "https://i.ytimg.com/vi/0ON22otQYVc/default.jpg",
            "width": 120,
            "height": 90
          },
          "medium": {
            "url": "https://i.ytimg.com/vi/0ON22otQYVc/mqdefault.jpg",
            "width": 320,
            "height": 180
          },
          "high": {
            "url": "https://i.ytimg.com/vi/0ON22otQYVc/hqdefault.jpg",
            "width": 480,
            "height": 360
     

In [49]:
itemRows = []

for item in videoJson['items']:
    itemRow = {'id': item['id']}
    for snippetKey in item['snippet'].keys():
        itemRow[snippetKey] = str(item['snippet'][snippetKey])
    itemRow['duration'] = item['contentDetails']['duration']
    itemRow['licensedContent'] = item['contentDetails']['licensedContent']

    
    itemRows.append(itemRow)
    
itemDf = pd.DataFrame(itemRows)[['id', 'publishedAt', 'title', 'duration']].sort_values('publishedAt', ascending=False)

itemDf

,id,publishedAt,title,duration
23,1V9TfNYeGhs,2021-08-18T13:46:45Z,بی بی سی اۉزبېک دستوری,PT15M16S
13,38zhgxnl5yk,2021-08-17T13:47:22Z,بی بی سی اۉزبېک دستوری,PT15M16S
6,hZPv3LgYzlE,2021-08-13T13:50:00Z,عیاللرگه قرشی سوقصد حاللری,PT15M16S
42,cGUWLtc4rGg,2021-07-16T13:47:20Z,اوروش شبرغان شهر آستانه‌سیگه یېتیب کېلدی,PT15M20S
17,nzPzikCcmcc,2021-07-09T13:48:35Z,طالبان مسکوده,PT15M16S
44,tZ4iS36CWqM,2021-07-05T13:46:43Z,طالبان برچه‌ خارجی کوچلرنینگ چیقیشینی ایسته‌گن,PT15M15S
27,l2XWaFnqoio,2021-06-29T13:46:57Z,شمالده گی خوفسیزلیک وضعیتی محاکمه قیلیندی,PT15M15S
39,CFZdI61YEHQ,2021-06-24T13:48:22Z,افغانستانده جنگلر دام اېتماقده,PT15M16S
31,7eZq5dYR_ss,2021-06-23T13:58:47Z,بیرلشگن ملتلر تشکیلاتی فاجعه‌لی یکوندن آگاه‌لن...,PT15M20S
43,Cq7VKD5Bmkc,2021-06-21T13:47:13Z,طالبان آله‌یاتگن تومنلر سانی آرتماقده,PT15M19S
